In [4]:
import numpy as np
from pylorentz import Momentum4
import pandas as pd
import os 
import uproot
import matplotlib.pyplot as plt

path = '../..//data/SR_nocut/'
files = os.listdir(path)

#Calculate significance
def calculate_significance(S, S_er, B, B_er):
    Z = np.sqrt(2 * ((S + B) * np.log(1 + (S / B)) - S))
    dZ_dS = np.log(1 + S/B) / (-2*S + 2 * (B + S) * np.log(1 + S/B))
    dZ_dB = (-S + B * np.log(1 + S/B)) / (B * np.sqrt(-2*S + 2 * (B + S) * np.log(1 + S/B)))
    Z_er = np.sqrt((dZ_dS * S_er)**2 + (dZ_dB * B_er)**2)

    return Z, Z_er

In [ ]:
llvv_qq = '345666'
llvv_gg = '345723'
llvvjj = '363724'

list_qq = []
list_gg = []
list_EWK = []
list_qq_alt = []
gw_cols = []
parameters = [
    'dLepR',
    'dMetZPhi',
    'met_tst',
    'MetOHT',
    'M2Lep',
    'global_weight'
    ]

for file in files:

    df = uproot.open(path + file)['tree'].arrays(library = 'pd')

    if '345666' in file:   
       print('Appending in QCD qq')
       list_qq.append(df)
    elif '345723' in file:
        print('Appending in QCD gg')
        list_gg.append(df)
    elif '363724' in file:
        print('Appending in EWK')
        list_EWK.append(df)
        # df = uproot.open(path = file)
    elif '361604' in file:
        print('Appending in QCD qq alternative')
        list_qq_alt.append(df)

frame_qq = pd.concat(list_qq,axis=0)
frame_gg = pd.concat(list_gg,axis=0)
frame_EWK = pd.concat(list_EWK,axis=0)
# frame_qq_alt = pd.concat(list_qq_alt,axis=0)


In [9]:
signal_qq = frame_qq['global_weight'].sum()
signal_qq_er = np.sqrt((frame_qq['global_weight']**2).sum())

signal_gg = frame_gg['global_weight'].sum()
signal_gg_er = np.sqrt((frame_gg['global_weight']**2).sum())

signal_EWK = frame_EWK['global_weight'].sum()
signal_EWK_er = np.sqrt((frame_EWK['global_weight']**2).sum())

signal_total = signal_qq + signal_gg + signal_EWK
signal_total_er = np.sqrt(signal_qq_er**2 + signal_gg_er**2 + signal_EWK_er**2)

sig_reco = 1577
sig_reco_er = 13.8
sig_fid = signal_total
sig_fid_er = signal_total_er

CZZ =  sig_reco / sig_fid
CZZ_er = np.sqrt((sig_reco_er / sig_fid) ** 2 + ( sig_reco * sig_fid_er / sig_fid ** 2) ** 2)

fb = 1
lumi = 140.1 / fb
lumi_er = 0.017 * lumi

z = lumi* CZZ
z_er = np.sqrt((CZZ * lumi_er) ** 2 + (lumi * CZZ_er) ** 2)

sigma_fid = sig_reco / z
sigma_fid_er = np.sqrt((sig_reco_er / z) ** 2 + ( sig_reco * z_er / z ** 2) ** 2)


 
print(f"Signal QCD qq =          {signal_qq:.2f} ± {signal_qq_er:.2f}")
print(f"Signal QCD gg =          {signal_gg:.2f} ± {signal_gg_er:.2f}")
print(f"Signal EWK =             {signal_EWK:.2f} ± {signal_EWK_er:.2f}")
print(f"Signal =                 {signal_total:.2f} ± {signal_total_er:.2f}")
print(f"Integrated Luminosity =  {lumi:.2f} ± {lumi_er:.2f} fb^{-1}")
print(f"Correction Factor =      {CZZ:.2f} ± {CZZ_er:.2f}")
print(f"Fiducial cross section = {sigma_fid:.2f} ± {sigma_fid_er:.2f} fb")

Signal QCD qq =          2032.64 ± 17.72
Signal QCD gg =          220.83 ± 0.33
Signal EWK =             4.30 ± 0.04
Signal =                 2257.77 ± 17.72
Integrated Luminosity =  140.10 ± 2.38 fb^-1
Correction Factor =      0.70 ± 0.01
Fiducial cross section = 16.12 ± 0.36 fb
